In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation.utils import GenerationConfig

/root/miniconda3/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/root/miniconda3/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
print(torch.cuda.is_available())

True


In [3]:
tokenizer = AutoTokenizer.from_pretrained("model/Baichuan2-13B-Chat", use_fast=False, trust_remote_code=True)

In [4]:
model = AutoModelForCausalLM.from_pretrained("model/Baichuan2-7B-Chat", device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True,offload_folder = "./Baichuan2/offload")
# model = model.quantize(8).cuda() #量化部署

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [5]:
model.generation_config = GenerationConfig.from_pretrained("model/Baichuan2-13B-Chat")
messages = []
messages.append({"role": "user", "content": "你好"})
response = model.chat(tokenizer, messages)
print(response)

OutOfMemoryError: CUDA out of memory. Tried to allocate 5.00 GiB. GPU 

In [ ]:
import requests

In [ ]:
from langchain.llms.base import LLM
from typing import Any, List, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import torch

class baichuan2_LLM(LLM):
    # 基于本地 Baichuan 自定义 LLM 类
    tokenizer : AutoTokenizer = None
    model: AutoModelForCausalLM = None

    def __init__(self, model_path :str):
        # model_path: Baichuan-13B-chat模型路径
        # 从本地初始化模型
        super().__init__()
        print("正在从本地加载模型...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True,offload_folder = "./Baichuan2/offload")
        self.model.generation_config = GenerationConfig.from_pretrained(model_path)
        self.model = self.model.eval()
        print("完成本地模型的加载")

    def _call(self, prompt : str, stop: Optional[List[str]] = None,
                run_manager: Optional[CallbackManagerForLLMRun] = None,
                **kwargs: Any):
         # 重写调用函数
        messages = [
            {"role": "user", "content": prompt}
        ]
         # 重写调用函数
        response= self.model.chat(self.tokenizer, messages)
        return response
        
    @property
    def _llm_type(self) -> str:
        return "baichuan2_LLM"

In [6]:
llm = baichuan2_LLM("./model/Baichuan2-13B-Chat")

NameError: name 'baichuan2_LLM' is not defined

In [8]:
llm ("你好")

NameError: name 'llm' is not defined

In [7]:
llm("苗族古歌分布在哪些地方")

NameError: name 'llm' is not defined